In [1]:
import pandas as pd
from assign_unique_ids_functions import *

In [28]:
prof = pd.read_csv('officer-profiles.csv.gz')
ref = pd.read_csv('officer-reference.csv.gz')

/Users/invinst/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3,10,11,12,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
test_uid = prof[(prof['First.Name'] == 'MICHELLE') & (prof['Last.Name'] == 'WILSON')]['UID']

In [32]:
prof[prof['UID'].isin(test_uid)]['Current.Unit']

31092    120.0
Name: Current.Unit, dtype: float64

In [38]:
test_uid
col = 'Current.Unit'
uid = 'UID'
mode_cols = [col]
id_cols=[]

In [17]:
df = ref[ref[uid].isin(test_uid)]

In [33]:
aggregate_data(df, uid, [],mode_cols=['Current.Unit'])

,UID,Current.Unit
0,31093,120.0


In [18]:
dfu = df[[uid, col]]
dfu

,UID,Current.Unit
31092,31093,NaN
63098,31093,NaN
95104,31093,720.0
126710,31093,120.0
132277,31093,123.0
146025,31093,123.0
148882,31093,NaN
164631,31093,NaN
191545,31093,NaN


In [19]:
dfu.dropna(axis=0, how='any', inplace=True)
dfu

/Users/invinst/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,UID,Current.Unit
95104,31093,720.0
126710,31093,120.0
132277,31093,123.0
146025,31093,123.0


In [20]:
kd_df = keep_duplicates(dfu, uid)
kd_df

,UID,Current.Unit
95104,31093,720.0
126710,31093,120.0
132277,31093,123.0
146025,31093,123.0


In [21]:
dfu = dfu[~dfu[uid].isin(kd_df[uid])]
dfu

,UID,Current.Unit


In [27]:
groups = kd_df.groupby(uid, as_index=False)
groups = [[k, stats.mode(g[col], nan_policy='propagate').mode[0]] for k, g in groups]
groups

[[31093, 123.0]]

In [34]:
# Original Code
def aggregate_data2(df, uid, id_cols=[],
                   mode_cols=[], max_cols=[],
                   current_cols=[], time_col=""):
    '''returns an aggregated pandas dataframe
       with one entry per specified uid (and id_cols combination)
       columns specified for aggregation can be aggregated by
       mode (finding most common value in a column for each uid),
       max (finding largest value in a column for each uid),
       or current (finding most recent value in the column
       using order_aggregate and using a specified time_col for ordering)

       EX: aggregate_data(..., 'A', ['B'], ['C'], ['D'])
       A B C D    A B C D
       1 3 1 2    1 3 1 3
       1 3 1 2 ->
       1 3 1 0
       1 3 5 3
    '''
    uid_col = [uid]  # Create list of specified uid
    # Initialize agg_df by taking unique rows (uids and id_cols)
    agg_df = df[uid_col + id_cols].drop_duplicates()
    agg_df.reset_index(drop=True, inplace=True)  # Reset index

    # Iterate over mode and max columns
    for col in mode_cols + max_cols:
        # Initialize unique dataframe with uids and specified column only
        dfu = df[[uid, col]].drop_duplicates()
        # Drop any rows with missing data
        dfu.dropna(axis=0, how='any', inplace=True)
        # Initialize dataframe of duplicates
        kd_df = keep_duplicates(dfu, uid)
        # Drop duplicate rows from dfu
        dfu = dfu[~dfu[uid].isin(kd_df[uid])]

        # If duplciate dataframe is empty
        if kd_df.empty:
            # No aggregation is neccessary on dfu since all values are unique
            # Merge dfu back to agg_df
            agg_df = agg_df.merge(dfu, on=uid, how='left')
        # If duplicate dataframe is not empty
        else:
            # Group duplicate dataframe by uid
            groups = kd_df.groupby(uid, as_index=False)
            # If col is specified in mode_cols
            if col in mode_cols:
                # Use list comprehension to generate a list of
                # two item lists: [uid, most common value in group]
                groups = [[k,
                           stats.mode(g[col],
                                      nan_policy='propagate').mode[0]]
                          for k, g in groups]
                # Recombine list of lists into dataframe
                groups = pd.DataFrame(groups, columns=[uid, col])

            # If col is specified in max_cols
            if col in max_cols:
                # Take max of each group, excluding NaNs
                groups = groups.agg(np.nanmax)

            # Append groups to dfu and merge to agg_df on uid
            agg_df = agg_df.merge(dfu.append(groups),
                                  on=uid, how='left')

    # If current_cols and time_col are specified
    if current_cols and time_col:
        # Ensure that time_col is a pandas datetime object
        df[time_col] = pd.to_datetime(df[time_col])
        # Aggregate data by taking most recent (by time_col)
        # occurence of the current_cols specified
        oa_df = order_aggregate(df[uid_col + [time_col] + current_cols],
                                uid_col, current_cols, [time_col])
        # Merge order aggregated data to agg_df on uid
        agg_df = agg_df.merge(oa_df, on=uid, how='left')
        # Now that the current_cols are 'current', the name must be changed
        # Add the prefix 'Current' to the current_cols in agg_df
        agg_df.columns = ['Current.' + col
                          if col in current_cols else col
                          for col in agg_df.columns]

    return agg_df

In [35]:
aggregate_data2(df, uid, [],mode_cols=['Current.Unit'])

,UID,Current.Unit
0,31093,120.0


In [39]:
uid_col = [uid]  # Create list of specified uid
# Initialize agg_df by taking unique rows (uids and id_cols)
agg_df = df[uid_col + id_cols].drop_duplicates()
agg_df.reset_index(drop=True, inplace=True)  

In [55]:
df = ref
for col in mode_cols:
        # Initialize unique dataframe with uids and specified column only
        print('1')
        print(df[[uid, col]])
        # I've been dropping duplicates..... sigh........
        dfc = df[[uid, col]]
        print('2')
        print(dfc)
        # Drop any rows with missing data
        dfc.dropna(axis=0, how='any', inplace=True)
        # Initialize dataframe of duplicates
        kd_df = keep_duplicates(dfc, uid)
        # Drop duplicate rows from dfu
        dfc = dfc[~dfc[uid].isin(kd_df[uid])]
        print('3')
        print(dfc)
        print(kd_df)
        # If duplciate dataframe is empty
        if kd_df.empty:
            # No aggregation is neccessary on dfu since all values are unique
            # Merge dfu back to agg_df
            agg_df = agg_df.merge(dfu, on=uid, how='left')
            print('Here')
        # If duplicate dataframe is not empty
        else:
            print('Nope, here')
            print(kd_df)
            # Group duplicate dataframe by uid
            groups = kd_df.groupby(uid, as_index=False)
            # If col is specified in mode_cols
            if col in mode_cols:
                # Use list comprehension to generate a list of
                # two item lists: [uid, most common value in group]
                groups = [[k,
                           stats.mode(g[col],
                                      nan_policy='propagate').mode[0]]
                          for k, g in groups]
                print(groups)
                # Recombine list of lists into dataframe
                groups = pd.DataFrame(groups, columns=[uid, col])

1
          UID  Current.Unit
0           1           NaN
1           2           NaN
2           3           NaN
3           4           NaN
4           5           NaN
5           6           NaN
6           7           NaN
7           8           NaN
8           9           NaN
9          10           NaN
10         11           NaN
11         12           NaN
12         13           NaN
13         14           NaN
14         15           NaN
15         16           NaN
16         17           NaN
17         18           NaN
18         19           NaN
19         20           NaN
20         21           NaN
21         22           NaN
22         23           NaN
23         24           NaN
24         25           NaN
25         26           NaN
26         27           NaN
27         28           NaN
28         29           NaN
29         30           NaN
...       ...           ...
194898  32204           NaN
194899  31522           NaN
194900  31536           NaN
194901  31555     

/Users/invinst/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


[[1, 3.0], [2, 15.0], [3, 543.0], [4, 20.0], [5, 640.0], [6, 25.0], [7, 16.0], [8, 132.0], [9, 192.0], [10, 151.0], [11, 606.0], [12, 50.0], [13, 50.0], [14, 630.0], [15, 6.0], [16, 191.0], [17, 1.0], [18, 19.0], [19, 19.0], [20, 17.0], [21, 640.0], [22, 4.0], [23, 142.0], [24, 44.0], [25, 16.0], [26, 8.0], [27, 11.0], [29, 701.0], [30, 50.0], [31, 72.0], [32, 13.0], [33, 630.0], [34, 1.0], [35, 6.0], [36, 24.0], [37, 44.0], [38, 277.0], [40, 24.0], [41, 50.0], [42, 24.0], [43, 8.0], [44, 11.0], [45, 17.0], [46, 15.0], [47, 18.0], [49, 10.0], [50, 16.0], [51, 11.0], [52, 15.0], [53, 6.0], [54, 10.0], [55, 14.0], [56, 10.0], [57, 384.0], [58, 1.0], [59, 10.0], [60, 17.0], [61, 44.0], [62, 19.0], [63, 13.0], [64, 376.0], [65, 17.0], [66, 1.0], [67, 24.0], [68, 4.0], [69, 44.0], [70, 4.0], [71, 701.0], [72, 13.0], [73, 20.0], [74, 44.0], [75, 50.0], [76, 9.0], [77, 44.0], [78, 45.0], [79, 18.0], [80, 7.0], [81, 59.0], [82, 17.0], [83, 313.0], [84, 610.0], [85, 86.0], [86, 168.0], [87, 173

In [56]:
groups

,UID,Current.Unit
0,1,3.0
1,2,15.0
2,3,543.0
3,4,20.0
4,5,640.0
5,6,25.0
6,7,16.0
7,8,132.0
8,9,192.0
9,10,151.0


In [70]:
def mode_aggregate(df, uid, col):
    # Initialize dataframe for specifically this column
    dfc = df[[uid, col]]
    # Drop null values
    dfc.dropna(axis=0, how='any', inplace=True)
    # Initialize dataframe of uids which have multiple values
    kd_df = keep_duplicates(dfc, uid)
    # If duplicates dataframe is empty
    if kd_df.empty:
        # Aggregation is not neccessary since each uid only has one row 
        return dfc # Return dfc
    # If there are duplicates
    else: 
        # Ensure dfc only contains rows with uids that only appear once
        dfc = dfc[~dfc[uid].isin(kd_df[uid])]
        # Group duplicate dataframe by uid
        groups = kd_df.groupby(uid, as_index=False)
        # If col is specified in mode_cols
        # Use list comprehension to generate a list of
        # two item lists: [uid, most common value in group]
        groups = [[k,
                    stats.mode(g[col],
                              nan_policy='propagate').mode[0]]
                  for k, g in groups]
        # Recombine list of lists into dataframe
        groups = pd.DataFrame(groups, columns=[uid, col])
        # Return dataframe of unique uids appended to aggregated uid data
        return dfc.append(groups)

In [ ]:
def max_aggregate(df, uid, col):
    # Initialize unique dataframe with uids and specified column only
    dfu = df[[uid, col]].drop_duplicates()
    # Drop any rows with missing data
    dfu.dropna(axis=0, how='any', inplace=True)
    # Initialize dataframe of duplicates
    kd_df = keep_duplicates(dfu, uid)
    # If duplciate dataframe is empty
    if kd_df.empty:
        # No aggregation is neccessary on dfu since all values are unique
        return dfu # Return unique dataframe
    # If duplicate dataframe is not empty
    else:
        # Drop duplicate rows from dfu
        dfu = dfu[~dfu[uid].isin(kd_df[uid])]
        # Group duplicate dataframe by uid
        groups = kd_df.groupby(uid, as_index=False)
        # Take max of each group, excluding NaNs
        groups = groups.agg(np.nanmax)
        # Return unique uid data appended to aggregated uid data
        dfu.append(groups)

In [71]:
# New Code
def aggregate_data3(df, uid, id_cols=[],
                   mode_cols=[], max_cols=[],
                   current_cols=[], time_col=""):
    '''returns an aggregated pandas dataframe
       with one entry per specified uid (and id_cols combination)
       columns specified for aggregation can be aggregated by
       mode (finding most common value in a column for each uid),
       max (finding largest value in a column for each uid),
       or current (finding most recent value in the column
       using order_aggregate and using a specified time_col for ordering)

       EX: aggregate_data(..., 'A', ['B'], ['C'], ['D'])
       A B C D    A B C D
       1 3 1 2    1 3 1 3
       1 3 1 2 ->
       1 3 1 0
       1 3 5 3
    '''
    uid_col = [uid]  # Create list of specified uid
    # Initialize agg_df by taking unique rows (uids and id_cols)
    agg_df = df[uid_col + id_cols].drop_duplicates()
    agg_df.reset_index(drop=True, inplace=True)  # Reset index

    # Iterate over mode cols
    for col in mode_cols:
        # Merge mode_aggregated column to agg_df
        agg_df = agg_df.merge(mode_aggregate(df[[uid, col]], uid, col),
                              on = uid, how = 'left')
    # Iterate over max cols
    for col in max_cols:
        # Merge max_aggregated column to agg_df
        agg_df = agg_df.merge(max_aggregate(df[[uid, col]], uid, col),
                              on = uid, how = 'left')

    # If current_cols and time_col are specified
    if current_cols and time_col:
        # Ensure that time_col is a pandas datetime object
        df[time_col] = pd.to_datetime(df[time_col])
        # Aggregate data by taking most recent (by time_col)
        # occurence of the current_cols specified
        oa_df = order_aggregate(df[uid_col + [time_col] + current_cols],
                                uid_col, current_cols, [time_col])
        # Merge order aggregated data to agg_df on uid
        agg_df = agg_df.merge(oa_df, on=uid, how='left')
        # Now that the current_cols are 'current', the name must be changed
        # Add the prefix 'Current' to the current_cols in agg_df
        agg_df.columns = ['Current.' + col
                          if col in current_cols else col
                          for col in agg_df.columns]

    return agg_df

In [72]:
t1 = aggregate_data3(df, uid, [],mode_cols=['Current.Unit'])

In [61]:
t2 = aggregate_data(df, uid, [], max_cols=['Current.Unit'])

In [66]:
t2[t2['UID'].isin(test_uid)]

,UID,Current.Unit
31092,31093,720.0
